<center style><h3>Computer Assignment 2</h3></center>
<center style><h4>Genetics</h4></center>
 <h4 style="text-align:right">
        محمد پویا افشاری - 810198577
</h4>

GA are a class of algorithms based on the abstraction of Darwin’s evolution of biological systems, pioneered by J. Holland and his collaborators in the 1960s and 1970s (Holland, 1975). GA use the genetic operators: crossover, mutation, and selection. Each solution is encoded as a string (often binary or decimal), called a chromosome. The crossover of two parent strings produces offspring (new solutions) by swapping parts or genes of the chromosomes. Crossover has a higher probability, typically 0.8-0.95.  This mutation probability is typically low, from 0.001 to 0.05.
New solutions generated in each generation will be evaluated by their fitness that is linked to the objective function of the optimization problem. The new solutions are selected according to their fitness—selection of the fittest. Sometimes, in order to make sure that the best solutions remain in the population, the best solutions are passed on to the next generation without much change. This is called elitism.

In [73]:
import random
import pandas as pd 
import numpy as np


In [74]:
# load data
df = pd.read_csv('sample.csv')


print(df.head())

   Unnamed: 0 ticker      risk    return
0           0    LMB  0.665555  2.459677
1           1  GECCM  0.011733  0.202107
2           2   OXBR  0.810532  2.040541
3           3   LIVE  0.630844  3.729542
4           4   NVEE  0.330513  1.610970


* Gene: A gene in this case will represent the proportion of capital invested in a specific stock. Each gene will be a floating-point number between 0 and 1, where the sum of all genes in a chromosome will always equal 1.

* Chromosome: A chromosome will represent a proposed solution to the problem of maximizing return while minimizing risk. Each chromosome will be a list of genes representing the proportion of capital invested in each stock.

* Population: The population will be a list of chromosomes representing multiple proposed solutions to the problem. Each chromosome in the population will be randomly generated initially, and then evolved over multiple generations using genetic operators such as selection, crossover, and mutation.

Mutation : This code first mutates a randomly selected gene in the chromosome by replacing it with a new gene. Then, it normalizes the chromosome by dividing each gene by the sum of all genes in the chromosome, if the sum is greater than 1. This ensures that the sum of genes in the chromosome is always less or equal to 1.

In [75]:

# Define the number of genes (stocks) and chromosomes (solutions) in the population
numGenes = 400
numChromosomes = 100
# define the hyperparameters for the mutation and crossover functions
mutation_rate = 0.001
crossover_rate = 0.8

# Define the gene as a floating-point number between 0 and 1
def createGene():
    gene = round(random.uniform(0, 1), 2)
#    print(f"Gene created: {gene}")
    return gene



# Define the chromosome as a list of genes where the sum of all genes equals 1
def createChromosome():
    chromosome = []
    sumGenes = 0
    for i in range(numGenes):
        gene = createGene()
        chromosome.append(gene)
        sumGenes += gene
    # Normalize the genes so that the sum equals 1
    for i in range(numGenes):
        chromosome[i] /= sumGenes
#    print(f"Chromosome created: {chromosome}")
    return chromosome

# Create an initial population of chromosomes randomly
def createPopulation():
    population = []
    for i in range(numChromosomes):
        chromosome = createChromosome()
        population.append(chromosome)
#    print(f"Population created: {population}")
    return population

# define fitness function
def fitness(chromosome):
    # calculate return and risk
    total_return = 0
    total_risk = 0
    for i in range(numGenes):
        total_return += chromosome[i] * df['return'][i]
        total_risk += chromosome[i] * df['risk'][i]
    # calculate number of shares
    num_shares = sum([1 for i in range(numGenes) if chromosome[i] > 0])
    # combine into fitness score
    return (5*total_return - 4*total_risk) * num_shares, num_shares



# Perform tournament selection to select the best parent chromosome
def tournamentSelection(population):
    # randomly select a subset of chromosomes from the population
    tournament = random.sample(population, k=5)
    # sort the tournament by fitness in descending order
    tournament.sort(key=lambda chromosome: fitness(chromosome)[0], reverse=True)
    # return the best chromosome as the parent
    parent = tournament[0]
#    print(f"Parent selected: {parent}")
    return parent

# Perform single point crossover to create two offspring chromosomes
def singlePointCrossover(parent1, parent2):
    # randomly select a crossover point
    crossover_point = random.randint(0, numGenes-1)
    # create offspring chromosomes by swapping genes from the parents
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
#    print(f"Parents: {parent1}, {parent2}")
#    print(f"Offspring created: {offspring1}, {offspring2}")
    return offspring1, offspring2


# Perform crossover on the current population
def crossover(population):
    new_population = []
    for i in range(numChromosomes // 2):
        # select two parent chromosomes using tournament selection
        parent1 = tournamentSelection(population)
        parent2 = tournamentSelection(population)
        # perform crossover to create two offspring chromosomes
        offspring1, offspring2 = singlePointCrossover(parent1, parent2)
        # append the offspring chromosomes to the new population
        new_population.append(offspring1)
        new_population.append(offspring2)
#        print(f"Offspring created: {offspring1}, {offspring2}")
    return new_population

# Define the mutation function
def mutate(chromosome):
    # randomly select a gene to mutate
    gene_index = random.randint(0, numGenes-1)
    
    # randomly generate a new gene
    new_gene = createGene()
    
    # replace the old gene with the new gene
    chromosome[gene_index] = new_gene
    
    # normalize the chromosome
    sum_genes = sum(chromosome)
    if sum_genes > 1:
        chromosome = [gene/sum_genes for gene in chromosome]
    
    return chromosome



def geneticAlgorithm():
    # create initial population
    population = createPopulation()
    
    # iterate through generations
    for generation in range(10):
        print(f"Generation: {generation}")
        # perform crossover on the population
        new_population = crossover(population)
        
        # mutate the new population
        for i in range(len(new_population)):
            new_population[i] = mutate(new_population[i])

                    
        # evaluate fitness of the new population
        fitness_scores = []
        for chromosome in new_population:
            fitness_score = fitness(chromosome)
            fitness_scores.append(fitness_score)
        
        # sort the new population by fitness in descending order
        new_population = [chromosome for _,chromosome in sorted(zip(fitness_scores,new_population), reverse=True)]
        
        # select the top 100 chromosomes to be the next generation
        population = new_population[:numChromosomes]
        
    # calculate and print the sum of risk and return for the best chromosome
    best_chromosome = max(population, key=lambda chromosome: fitness(chromosome)[0])
    best_return = sum([best_chromosome[i] * df['return'][i] for i in range(numGenes)])
    best_risk = sum([best_chromosome[i] * df['risk'][i] for i in range(numGenes)])
    Sum = sum([best_chromosome[i] for i in range(numGenes)]
              )
    print(f"Best chromosome: {best_chromosome}")
    print(f"Sum of return for the best chromosome: {best_return}")
    print(f"Sum of risk for the best chromosome: {best_risk}")
    print(f"Sum of values(shares):{Sum}")
    return best_chromosome




# create initial population
population = createPopulation()
# perform crossover on the population
new_population = crossover(population)
# print the new population
#print(f"New population: {new_population}")


createPopulation()
chromosome = createChromosome()
# perform crossover on the population
new_population = crossover(population)
#print("Chromosome: ", chromosome)
#print("Fitness: ", fitness(chromosome))
# print the new population
#print(f"New population: {new_population}")

solution = geneticAlgorithm() 
#solution[:] = [x / 100 for x in solution]
print(solution)

# Read the CSV file into a DataFrame
df = pd.read_csv('sample.csv')
df['value'] = solution 
df.to_csv('updated_sample.csv', index=False)


Generation: 0
Generation: 1
Generation: 2
Generation: 3
Generation: 4
Generation: 5
Generation: 6
Generation: 7
Generation: 8
Generation: 9
Best chromosome: [0.00011891387928033228, 0.00015686511734852343, 2.5300825378794105e-05, 8.855288882577935e-05, 0.00013156429196972932, 0.0001897561903409558, 0.00016445536496216163, 0.00024035784109854397, 0.00011132363166669404, 2.024066030303528e-05, 5.060165075758821e-05, 0.00012903420943184989, 9.361305390153814e-05, 0.0, 0.00014927486973488518, 6.072198090910583e-05, 7.590247613638229e-06, 5.060165075758821e-05, 0.00024035784109854397, 0.0002201171807955087, 1.7303332139471193e-05, 0.0, 1.2977499104603398e-05, 5.7677773798237336e-05, 1.4419443449559334e-05, 0.00011968138063134245, 0.00013265887973594584, 0.00010526193718178309, 1.2977499104603398e-05, 0.00010670388152673903, 8.219082766248818e-05, 0.00013265887973594584, 7.209721724779667e-06, 0.00012112332497629837, 9.084249373222379e-05, 0.00011968138063134245, 0.00011102971456160684, 0.00

### Questions:

1. What problems do a very small or very large initial population cause?

A very small initial population can lead to a lack of diversity in the population and may result in the algorithm getting stuck in a local optima.

A very large initial population can be computationally expensive and may take longer to converge to an optimal solution.

2. If the number of population increases in each period, what effect does it have on the accuracy and speed of the algorithm?

Increasing the number of population in each period can improve the accuracy of the algorithm by increasing the diversity of the population, but it can also slow down the algorithm due to the increased computation required to evaluate the fitness of a larger population.

3. State and compare the effect of each crossover and mutation operation. Is it possible to use only one of them? Why ?

- Crossover: Swaping parts of the solution with another in chromosomes or solution representations. The main role is to provide mixing of the solutions and convergence in a subspace.

- Mutation: The change of parts of one solution randomly, which increases the diversity of the population and provides a mechanism for escaping from a local optimum.

It is possible to use only one of them, but using both crossover and mutation generally leads to better results as they complement each other in terms of exploring the search space and maintaining diversity in the population.

4. In your opinion, what solutions are there to get a faster solution to this particular problem?

- One solution to get a faster solution to this particular problem could be to use a more advanced optimization algorithm such as a particle swarm optimization or a differential evolution algorithm. These algorithms are designed to handle high-dimensional optimization problems and can converge to optimal solutions faster than a genetic algorithm.
 
- Fine-tuning the algorithm parameters, such as population size, crossover and mutation rates, and selection methods, to achieve better performance.

- Implementing local search algorithms to improve the performance of the genetic algorithm, such as using hill-climbing to find better solutions around the current best solution.

5. Despite the use of these methods, it is still possible that the chromosomes will not change after a few more steps. Explain the reason for this and the problems it causes. What do you suggest to solve it?

The chromosomes may not change after a few more steps due to a phenomenon called premature convergence, where the algorithm becomes stuck in a suboptimal solution and is unable to escape it.

This can be caused by a too small population size or a lack of diversity in the population due to the selection of the same individuals in each generation. To solve this problem, some possible solutions include:

- Increasing the population size to maintain diversity and prevent premature convergence.

- Using different selection methods to avoid selecting the same individuals repeatedly.

- Using adaptive parameters to change the crossover and mutation rates dynamically based on the current state of the population.

6. What solution do you suggest to end the program if the problem is not solved?

If the problem is not solved within a certain number of generations or a given amount of time, the program can be terminated and the current best solution can be returned as the final solution.

Alternatively, we can set a threshold, the program can be stop when a certain fitness threshold is reached or when the improvement in fitness over a certain number of generations falls below a certain threshold. It is also possible to combine different stopping criteria to ensure that the program terminates when the desired level of performance is achieved.